In [ ]:
import datetime as dt
import utils
from sentinelsat import SentinelAPI
import json
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

from rasterio import plot

In [ ]:
geojson_path = "data/var.geojson"
dates = (dt.date(2021, 8, 14), dt.date(2021, 8, 25))
credentials_path = "secrets/sentinel_api_credentials.json"

with open(credentials_path, 'r') as infile:
    credentials = json.load(infile)

api = SentinelAPI(
    credentials["username"],
    credentials["password"]
)

In [ ]:
images_df = utils.get_dataframe_between_dates(api, dates[0], dates[1], geojson_path)

In [ ]:
key_columns = ["cloudcoverpercentage", "vegetationpercentage", "waterpercentage"]
images_df[key_columns]

In [ ]:
uuid, title = utils.minimize_dataframe(images_df)
print(uuid, title)

In [ ]:
path = './data/'
utils.download_from_api(api, uuid, title, path)

In [ ]:
im_path = path + title + '.SAFE'
output_folder = './output/'
print(im_path)
img_before = utils.create_tiff_image(im_path, 'before', 'B04', 'B08',
    output_folder=output_folder, resolution=10)

In [ ]:
corr_matrix = images_df.corr()

plt.figure(figsize=(10, 10))
hm = sns.heatmap(corr_matrix, linewidths=1, linecolor='gray', cmap='coolwarm')
hm.set_yticklabels(hm.get_yticklabels(), fontsize=11)
hm.set_xticklabels(hm.get_xticklabels(), rotation=45, fontsize=11, ha='right')
plt.show()

In [ ]:
image_path = '/'.join(
    ['./data', 'S2B_MSIL2A_20210822T102559_N0301_R108_T31TGJ_20210822T133528.SAFE', 'GRANULE',
    'L2A_T31TGJ_A023300_20210822T103247', 'IMG_DATA', 'R10m/']
)
print(image_path)

band4 = rasterio.open(image_path + 'T31TGJ_20210822T102559_B04_10m.jp2', driver='JP2OpenJPEG') # red
band8 = rasterio.open(image_path + 'T31TGJ_20210822T102559_B08_10m.jp2', driver='JP2OpenJPEG') # nir

print(band4.count)
print(band4.width)
print(band4.height)

red = band4.read(1).astype('float64')
nir = band8.read(1).astype('float64')

ndvi_bef = np.where(
    (nir + red) == 0.,
    0.,
    (nir - red) / (nir + red)
)

In [ ]:
plt.figure(figsize=(16, 9))
plot.show(ndvi_bef)
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
plot.show(img_before)
plt.show()

In [ ]:
wildfire_date = dt.date(2021, 8, 17)
observation_interval = 14

In [ ]:
before_image, after_image = utils.get_before_after_images(
    api, dt.date(2021, 8, 17), 14, geojson_path,
    "B04", "B08", cloud_threshold=40, resolution=10
)

In [ ]:
plt.imshow(before_image)
plt.show()